In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import math

C:\Users\GCC\AppData\Local\Temp\ipykernel_24700\3160102943.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
##절대경로 참조
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
from library.quant_functions import anal_funcs, data_funcs

In [3]:
import yfinance as yf
## FRED 정보 수집
from fredapi import Fred
fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')

In [4]:
# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [5]:
from scipy.signal import find_peaks
import scipy.fft
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from itertools import product

## Process

In [6]:
df = data_funcs.get_merged_df('^GSPC', '^IXIC', 'GLD')
DGS20 = fred.get_series('DGS20')
DGS10 = fred.get_series('DGS10')
DGS2 = fred.get_series('DGS2')
DTB3 = fred.get_series('DTB3')

df['DGS20'] = (-(DGS20.diff()*20*0.9*0.01) + 1).cumprod()
df['DGS10'] = (-(DGS10.diff()*10*0.9*0.01) + 1).cumprod()
df['DGS2'] = (-(DGS2.diff()*2*0.9*0.01) + 1).cumprod()
df['DTB3'] = (-(DTB3.diff()*0.25*0.9*0.01) + 1).cumprod()

[*********************100%%**********************]  3 of 3 completed


In [7]:
returns = df.pct_change()

In [8]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import lightgbm as lgb

In [80]:
_temp = returns.dropna(subset=['^IXIC']).dropna(axis=1)

In [87]:
X_col = _temp.columns.to_list()
y_col = X_col.pop(X_col.index("^IXIC"))

In [91]:
rf = RandomForestRegressor()
rf.fit(_temp[X_col], _temp[y_col])

RandomForestRegressor()

In [ ]:
rf.predict

In [95]:
returns[X_col][returns[y_col].isna()]

Ticker,^GSPC,DGS20,DGS10,DTB3
Date,,,,
1927-12-30,NaN,NaN,NaN,NaN
1928-01-03,0.005663,NaN,NaN,NaN
1928-01-04,-0.002252,NaN,NaN,NaN
1928-01-05,-0.009594,NaN,NaN,NaN
1928-01-06,0.006268,NaN,NaN,NaN
...,...,...,...,...
1971-02-01,0.005632,-0.0036,0.0000,0.000068
1971-02-02,0.000104,-0.0018,-0.0018,0.000045
1971-02-03,0.002074,0.0000,0.0000,0.000023
